In [1]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
from tabulate import tabulate
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

In [3]:
import json
with open('GA_output_ET.json', 'r') as fp:
    feature_list = json.load(fp)

In [4]:
feature_list

{'SYN': ['Src Port',
  'Flow Duration',
  'Tot Fwd Pkts',
  'Tot Bwd Pkts',
  'TotLen Fwd Pkts',
  'TotLen Bwd Pkts',
  'Fwd Pkt Len Min',
  'Fwd Pkt Len Std',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Std',
  'Flow Byts/s',
  'Flow IAT Mean',
  'Flow IAT Std',
  'Flow IAT Max',
  'Flow IAT Min',
  'Fwd IAT Tot',
  'Fwd IAT Mean',
  'Fwd IAT Std',
  'Fwd IAT Max',
  'Fwd IAT Min',
  'Bwd IAT Mean',
  'Bwd IAT Std',
  'Bwd IAT Max',
  'Bwd IAT Min',
  'Bwd Pkts/s',
  'Pkt Len Max',
  'Pkt Len Mean',
  'Pkt Len Std',
  'SYN Flag Cnt',
  'PSH Flag Cnt',
  'ACK Flag Cnt',
  'Pkt Size Avg',
  'Fwd Seg Size Avg',
  'Bwd Seg Size Avg',
  'Subflow Fwd Pkts',
  'Active Std',
  'Active Min',
  'Idle Mean',
  'Idle Std',
  'Idle Max',
  'Idle Min',
  'Label'],
 'HTTP': ['Dst Port',
  'Protocol',
  'Tot Fwd Pkts',
  'Tot Bwd Pkts',
  'TotLen Fwd Pkts',
  'TotLen Bwd Pkts',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Min',
  'Fwd Pkt Len Mean',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Min',
  'B

In [5]:
file_list={
"SYN":[ '../csvs\\dos-synflooding-1-dec.pcap_Flow.csv', '../csvs\\dos-synflooding-6-dec.pcap_Flow.csv',],
"HTTP":['../csvs\\mirai-httpflooding-4-dec.pcap_Flow.csv', '../csvs\\mirai-httpflooding-1-dec.pcap_Flow.csv'],
"ACK":['../csvs\\mirai-ackflooding-4-dec.pcap_Flow.csv','../csvs\\mirai-ackflooding-1-dec.pcap_Flow.csv'],
"UDP":[ '../csvs\\mirai-udpflooding-4-dec.pcap_Flow.csv','../csvs\\UDP_DDoS_00001.pcap_Flow.csv'],
"ARP":[ '../csvs\\mitm-arpspoofing-6-dec.pcap_Flow.csv', '../csvs\\mitm-arpspoofing-5-dec.pcap_Flow.csv'],
"SP":['../csvs\\scan-hostport-3-dec.pcap_Flow.csv','../csvs\\Edge Port Scanning attack.pcap_Flow.csv'],
"BF":[ '../csvs\\mirai-hostbruteforce-5-dec.pcap_Flow.csv','../csvs\\Edge Password attacks.pcap_Flow.csv']
}

In [6]:
def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

In [7]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

### RandomizedSearchCV  DT

In [8]:
lines=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))

    nfolds=10
    param_grid = { 'criterion':['gini','entropy'],
                      "max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                     "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                        # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                        "max_features" : sp_randint(1,X_train.shape[1])}

    second=time()
    f1=[]
    clf=DecisionTreeClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(50)):
        second=time()
        a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(10):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("DT_HPO.csv",index=False)

final_parametres=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9886387839098433   1.041    24      


100%|██████████| 50/50 [01:14<00:00,  1.48s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9943676873288527   0.439    24      


100%|██████████| 50/50 [00:12<00:00,  4.15it/s]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.906046015859736    0.486    24      


...

SP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.119    24      


100%|██████████| 50/50 [00:04<00:00, 10.19it/s]


BF
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.5625560748710202   1.253    24      


100%|██████████| 50/50 [01:56<00:00,  2.34s/it]

    criterion      max_depth    max_features    min_samples_split        F1         Std    Time    No  Attack
--  -----------  -----------  --------------  -------------------  --------  ----------  ------  ----  --------
 0  gini                  11              26                    7  0.991483  0            1.619    43  SYN
 1  gini                  11              19                    4  0.995777  0.00140916   0.248     0  HTTP
 2  entropy                5              24                    6  0.940911  0.00236887   0.352     3  ACK
 3  gini                   2               1                    8  0.054421  0.0222639    0.899    13  UDP
 4  entropy               25               9                    4  0.965713  0.0086615    0.063    15  ARP
 5  gini                   2              17                    7  1         0            0.101    13  SP
 6  gini                  11               8                    2  0.719363  0.160111     1.477    14  BF


### RandomizedSearchCV RF

In [ ]:
lines=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]



for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))
    # use a full grid over all parameters
    param_grid = {"max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                  "n_estimators" : sp_randint(1, 200),
                  "max_features": sp_randint(1, 11),
                  "min_samples_split":sp_randint(2, 11),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    second=time()
    f1=[]
    clf=RandomForestClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(RandomForestClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("RF_HPO.csv",index=False)

final_parametres=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]

for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9914825015895742   1.924    0       


100%|██████████| 10/10 [04:46<00:00, 28.61s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9943676873288524   0.124    0       


100%|██████████| 10/10 [00:18<00:00,  1.84s/it]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9824712973279108   0.124    0       


...

SP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.251    0       


100%|██████████| 10/10 [00:40<00:00,  4.01s/it]


BF
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.4371969877394523   1.014    0       


100%|██████████| 10/10 [12:53<00:00, 77.33s/it]

    bootst    criter      max_depth    max_features    min_samp_split    n_estimators         F1          Std    Time    No  Attack
--  --------  --------  -----------  --------------  ----------------  --------------  ---------  -----------  ------  ----  --------
 0  True      gini                3               8                 2             196  0.991483   0            39.8       7  SYN
 1  False     gini               18              10                 4               2  0.994931   0.00112733    1.039     6  HTTP
 2  True      entropy            31               9                 7              61  0.975863   0.00476664    2.213     4  ACK
 3  True      gini               13               7                 2             130  0.0503196  0.00852012   87.58      6  UDP
 4  False     gini               29               6                 2             151  0.985557   0.000990253   4.758     7  ARP
 5  True      entropy             7              10                 3             129  1

### XG Boost

In [19]:
from xgboost import XGBClassifier
clf = XGBClassifier(tree_method='gpu_hist')
print(clf.get_xgb_params())


{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'eval_metric': None, 'gamma': None, 'grow_policy': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'monotone_constraints': None, 'multi_strategy': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': 'gpu_hist', 'validate_parameters': None, 'verbosity': None}


In [20]:
import xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import PredefinedSplit
import sklearn.metrics
from tqdm import tqdm
from time import time
from tabulate import tabulate

lines = [['n_estimators', 'min_child_weight', 'max_depth', 'learning_rate', "F1", "Std", "Time", "No", "Attack"]]

for j in file_list:
    print(j)

    df = pd.read_csv(file_list[j][0], usecols=feature_list[j])
    X_train = df.iloc[:, 0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train = df['Label'].cat.codes

    df = pd.read_csv(file_list[j][1], usecols=feature_list[j])
    X_test = df.iloc[:, 0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test = df['Label'].cat.codes

    X = np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)

    param_grid = {
        'n_estimators': [100, 500, 900, 1100, 1500],
        'max_depth': [2, 3, 5, 10, 15],
        'learning_rate': [0.05, 0.1, 0.15, 0.20],
        'min_child_weight': [1, 2, 3, 4]
    }

    second = time()
    f1 = []
    clf = xgboost.XGBClassifier(tree_method='gpu_hist')  # Sử dụng GPU
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict = clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
    f1 = sum(f1) / len(f1)
    print('%-35s %-20s %-8s %-8s' % ("default", f1, round(time() - second, 3), ii))

    ######################################################################################################################
    for i in tqdm(range(5)):
        second = time()
        a, b, clf = run_random_search(xgboost.XGBClassifier(tree_method='gpu_hist'), param_grid, X, y)  # Sử dụng GPU
        f1 = []
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict = clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
        f1_result = sum(f1) / len(f1)
        f1 = np.array(f1)
        stndtd = f1.std()
        temp = list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a, f1_result, round(time() - second, 3), i))
        temp = temp + [f1_result, stndtd, round(time() - second, 3), i, j]
        lines.append(temp)

results = pd.DataFrame(lines[1:], columns=lines[0])
results.to_csv("XGB_HPO.csv", index=False)

print(tabulate(results, headers=list(results.columns)))

final_parametres = []
for i in results["Attack"].unique():
    df = results[results["Attack"] == i]
    m = df["F1"].max()
    df = df[df["F1"] == m]
    m = df["max_depth"].min()
    df = df[df["max_depth"] == m]
    final_parametres.append(list(df.values)[0])

results = pd.DataFrame(final_parametres[1:], columns=final_parametres[0])
print(tabulate(results, headers=list(results.columns)))


SYN
default                             0.9832474816849817   0.548    0       


 20%|██        | 1/5 [00:04<00:19,  4.88s/it]

{'n_estimators': np.int64(900), 'max_depth': np.int64(3), 'learning_rate': np.float64(0.1), 'min_child_weight': np.int64(2)} 0.965345567559568    4.876    0       


 40%|████      | 2/5 [00:09<00:14,  4.79s/it]

{'n_estimators': np.int64(900), 'max_depth': np.int64(15), 'learning_rate': np.float64(0.2), 'min_child_weight': np.int64(4)} 0.9559257770208716   4.726    1       


...

{'n_estimators': np.int64(1500), 'max_depth': np.int64(2), 'learning_rate': np.float64(0.1), 'min_child_weight': np.int64(3)} 0.8102097407180967   6.604    2       


 80%|████████  | 4/5 [00:19<00:04,  4.95s/it]

{'n_estimators': np.int64(900), 'max_depth': np.int64(2), 'learning_rate': np.float64(0.1), 'min_child_weight': np.int64(4)} 0.7905331214320429   3.915    3       


100%|██████████| 5/5 [00:27<00:00,  5.47s/it]

{'n_estimators': np.int64(1500), 'max_depth': np.int64(10), 'learning_rate': np.float64(0.15), 'min_child_weight': np.int64(3)} 0.7761652894385079   7.893    4       
      n_estimators    min_child_weight    max_depth    learning_rate          F1          Std    Time    No  Attack
--  --------------  ------------------  -----------  ---------------  ----------  -----------  ------  ----  --------
 0             900                   3         0.1                 2  0.965346    1.11022e-16   4.876     0  SYN
 1             900                  15         0.2                 4  0.955926    0             4.727     1  SYN
 2             500                   2         0.2                 3  0.955926    0             2.918     2  SYN
 3             900                   5         0.1                 2  0.965346    1.11022e-16   4.68      3  SYN
 4             900                   2         0.2                 1  0.983247    1.11022e-16   3.651     4  SYN
 5             100                

### RandomSearch CV Naive Bayes

In [21]:
from sklearn.naive_bayes import GaussianNB

In [22]:
lines=[['var_smoothing', "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
        'var_smoothing': np.logspace(0,-9, num=100),
    }

 
    second=time()
    f1=[]
    clf=GaussianNB()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(GaussianNB(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)


results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("NB_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    


SYN
default                             0.8602517092548031   0.023    0       


100%|██████████| 10/10 [00:00<00:00, 10.39it/s]


HTTP
default                             0.8477272914724974   0.004    0       


100%|██████████| 10/10 [00:00<00:00, 53.10it/s]


ACK
default                             0.44730103984837444  0.009    0       


...

SP
default                             0.9425323964186256   0.008    0       


100%|██████████| 10/10 [00:00<00:00, 22.57it/s]


BF
default                             0.3075579114185498   0.067    0       


100%|██████████| 10/10 [00:03<00:00,  3.14it/s]

      var_smoothing           F1          Std    Time    No  Attack
--  ---------------  -----------  -----------  ------  ----  --------
 0      0.000657933  0.801597     0             0.101     0  SYN
 1      1e-09        0.860252     1.11022e-16   0.103     1  SYN
 2      0.00231013   0.784943     0             0.09      2  SYN
 3      1            0.784943     0             0.082     3  SYN
 4      1            0.784943     0             0.103     4  SYN
 5      5.3367e-05   0.874485     0             0.092     5  SYN
 6      0.0351119    0.784943     0             0.082     6  SYN
 7      1            0.784943     0             0.082     7  SYN
 8      1.51991e-07  0.917438     1.11022e-16   0.133     8  SYN
 9      0.231013     0.784943     0             0.09      9  SYN
10      2.31013e-05  0.655534     0             0.018     0  HTTP
11      3.51119e-06  0.689046     0             0.021     1  HTTP
12      5.3367e-06   0.680837     0             0.019     2  HTTP
13      0.1519

### RandomSearch CV SVM

In [ ]:
lines = [["gamma", "C", "F1", "Std", "Time", "No", "Attack"]]

for j in file_list:
    print(j)

    df = pd.read_csv(file_list[j][0], usecols=feature_list[j])
    X_train = df.iloc[:, 0:-1]
    df["Label"] = df["Label"].astype("category")
    y_train = df["Label"].cat.codes

    df = pd.read_csv(file_list[j][1], usecols=feature_list[j])
    X_test = df.iloc[:, 0:-1]
    df["Label"] = df["Label"].astype("category")
    y_test = df["Label"].cat.codes

    X = np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [
        0 for _ in range(X_test.shape[0])
    ]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)

    param_grid = {"C": [0.001, 0.01, 0.1, 1, 10], "gamma": [0.001, 0.01, 0.1, 1]}

    second = time()
    f1 = []
    clf = svm.SVC()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict = clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
    f1 = sum(f1) / len(f1)
    # if f1>0.76:
    print("%-35s %-20s %-8s %-8s" % ("default", f1, round(time() - second, 3), ii))

    ######################################################################################################################
    for i in tqdm(range(1)):
        second = time()
        a, b, clf = run_random_search(svm.SVC(), param_grid, X, y)
        f1 = []
        for ii in range(1):
            clf.fit(X_train, y_train)
            predict = clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
        f1_result = sum(f1) / len(f1)
        f1 = np.array(f1)
        stndtd = f1.std()
        temp = list(a.values())
        print("%-90s %-20s %-8s %-8s" % (a, f1_result, round(time() - second, 3), i))
        temp = temp + [f1_result, stndtd, round(time() - second, 3), i, j]
        lines.append(temp)

        # if f1>0.76:

results = pd.DataFrame(lines[1:], columns=lines[0])
results.to_csv("svm_HPO.csv", index=False)


print(tabulate(results, headers=list(results.columns)))


SYN
default                             0.7849428130587054   0.212    0       


100%|██████████| 1/1 [00:35<00:00, 35.27s/it]


{'C': np.float64(1.0), 'gamma': np.float64(1.0)}                                           0.49871553348176056  35.272   0       
HTTP
default                             0.6062213740458016   0.078    0       


100%|██████████| 1/1 [00:00<00:00,  5.88it/s]


{'C': np.float64(0.1), 'gamma': np.float64(0.01)}                                          0.4403752865291327   0.169    0       
ACK
default                             0.06262248472042994  0.442    0       


...

{'C': np.float64(0.01), 'gamma': np.float64(0.1)}                                          0.27938671209540034  0.031    0       
SP
default                             0.9123699239911589   0.066    0       


100%|██████████| 1/1 [00:00<00:00,  7.49it/s]


{'C': np.float64(0.1), 'gamma': np.float64(1.0)}                                           0.49264996053670085  0.132    0       
BF
default                             0.3499427203203534   2.714    0       


100%|██████████| 1/1 [00:11<00:00, 11.81s/it]

{'C': np.float64(1.0), 'gamma': np.float64(0.1)}                                           0.3499427203203534   11.814   0       
      gamma      C           F1    Std    Time    No  Attack
--  -------  -----  -----------  -----  ------  ----  --------
 0    1      1      0.498716         0  35.272     0  SYN
 1    0.1    0.01   0.440375         0   0.169     0  HTTP
 2    0.01   0.001  0.0216884        0   0.708     0  ACK
 3    0.001  0.001  3.43299e-05      0   0.968     0  UDP
 4    0.01   0.1    0.279387         0   0.031     0  ARP
 5    0.1    1      0.49265          0   0.132     0  SP
 6    1      0.1    0.349943         0  11.814     0  BF
